## 재무정보(전체 재무제표) - 단일 회사(상장기업) - 사업보고서
+ 작성: 임경호
+ 금융감독원 OPEN DART https://opendart.fss.or.kr/


### 1. 대상 회사/연도/보고서 설정 및 재무제표 정보 가져오기

In [23]:
fiscal_year = '2020'
report_code = '11011'      # 사업보고서
listed_corp_codes = \
['01208885',    # 경동도시가스
 '00787376',    # 대성에너지
 '00128555',    # 삼천리
 '00130684']    # 서울도시가스

In [24]:
import requests
import pandas as pd
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [25]:
def dart_report_exist(cursor, report_code, fiscal_year, corp_code):
    query = f'SELECT EXISTS (SELECT * FROM dart_report_detail \
                              WHERE reprt_code = {report_code} and bsns_year = {fiscal_year} and corp_code = {corp_code})'

    cursor.execute(query)
    row = cursor.fetchone()
    data_exist = row[0]     # 저장된 데이터의 유무(1 - 데이터 있음)
    return data_exist

In [26]:
# 재무제표 가져오기
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
df_fi_all = pd.DataFrame()

for corp_code in listed_corp_codes:
    data_exist = dart_report_exist(cur, report_code, fiscal_year, corp_code)
    if data_exist == 1:     # 이미 데이터 수집/저장되었으면
        pass
    else:                   # 데이터 수집 필요
        params = {
            'corp_code': corp_code,
            'bsns_year': fiscal_year,
            'reprt_code': report_code,  
            'fs_div': 'OFS',            # OFS:재무제표
            'crtfc_key': api_key,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:             # URL GET 200 : 정상
            print("URL GET Error")
        if response.json()['status'] == '000':      # 데이터 유무 '000' : 정상
            # 데이터프레임 형태로 변환
            df_fi = pd.json_normalize(response.json()['list'])
            if not df_fi.empty:
                df_fi_all = pd.concat([df_fi_all, df_fi], ignore_index=True)

conn.close()

In [27]:
if not df_fi_all.empty:
    print(df_fi_all.size)
else:
    print("가져온 데이터가 없습니다.")

12240


In [28]:
df_fi_all['account_id'].str.len().max()

104

In [29]:
a = ['영업이익(손실)', '영업이익']
df_fi_all.query('account_nm in @a')

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
59,20210311001095,11011,2020,01208885,CIS,포괄손익계산서,dart_OperatingIncomeLoss,영업이익,-,제 4 기,17853641242,제 3 기,27681162391,제 2 기,43022491541,9,KRW,
231,20210319001077,11011,2020,00787376,CIS,포괄손익계산서,dart_OperatingIncomeLoss,영업이익(손실),-,제 12 기,24662747504,제 11 기,22731641468,제 10 기,15758300362,15,KRW,
369,20210318001068,11011,2020,00128555,CIS,포괄손익계산서,dart_OperatingIncomeLoss,영업이익,-,제 55 기,55054297793,제 54 기,55940933975,제 53 기,55291623589,10,KRW,
537,20210315000572,11011,2020,00130684,CIS,포괄손익계산서,dart_OperatingIncomeLoss,영업이익,-,제 38 기,6816926493,제 37 기,9810276797,제 36 기,2961784338,4,KRW,


In [30]:
div= 'SCE'
df_fi_all.query('sj_div == @div')

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
106,20210311001095,11011,2020,01208885,SCE,자본변동표,dart_EquityAtBeginningOfPeriod,기초자본,별도재무제표 [member],제 4 기,337701665705,제 3 기,318760537227,제 2 기,292628586274,1,KRW,NaN
107,20210311001095,11011,2020,01208885,SCE,자본변동표,dart_EquityAtBeginningOfPeriod,기초자본,자본 [member]|기타자본구성요소 [member],제 4 기,-177516494,제 3 기,-149680342,제 2 기,-149680342,1,KRW,NaN
108,20210311001095,11011,2020,01208885,SCE,자본변동표,dart_EquityAtBeginningOfPeriod,기초자본,자본 [member]|이익잉여금 [member],제 4 기,63556419764,제 3 기,44587455134,제 2 기,18455504181,1,KRW,NaN
109,20210311001095,11011,2020,01208885,SCE,자본변동표,dart_EquityAtBeginningOfPeriod,기초자본,자본 [member]|자본금 [member],제 4 기,14738515000,제 3 기,11792657500,제 2 기,11792657500,1,KRW,NaN
110,20210311001095,11011,2020,01208885,SCE,자본변동표,dart_EquityAtBeginningOfPeriod,기초자본,자본 [member]|자본잉여금 [member],제 4 기,259584247435,제 3 기,262530104935,제 2 기,262530104935,1,KRW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,20210315000572,11011,2020,00130684,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|기타자본항목,제 38 기,-45445180487,제 37 기,-45337353977,제 36 기,-45337353977,10,KRW,NaN
676,20210315000572,11011,2020,00130684,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|기타포괄손익누계액 [member],제 38 기,-4210882124,제 37 기,-3857988147,제 36 기,-2640391647,10,KRW,NaN
677,20210315000572,11011,2020,00130684,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|이익잉여금 [member],제 38 기,1012345856034,제 37 기,921030218206,제 36 기,914676820530,10,KRW,NaN
678,20210315000572,11011,2020,00130684,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|자본금 [member],제 38 기,35000000000,제 37 기,35000000000,제 36 기,35000000000,10,KRW,NaN


### 2. 데이터 전처리

In [31]:
if not df_fi_all.empty:
    # 불필요한 컬럼 삭제
    # df_fi_all.drop(labels=['account_id'], axis=1, inplace=True)   # 필수 항목. 삭제하면 안 됨. 코드 수정(2023.07.24)
    
    # 문자열 컬럼에 있는 null값을 ''로 변환
    df_fi_all['thstrm_nm'].fillna('', inplace=True)
    df_fi_all['frmtrm_nm'].fillna('', inplace=True)
    df_fi_all['bfefrmtrm_nm'].fillna('', inplace=True)

    # 금액 컬럼에 있는 null값을 ''로 변환
    df_fi_all['thstrm_amount'].fillna('', inplace=True)
    df_fi_all['frmtrm_amount'].fillna('', inplace=True)
    df_fi_all['bfefrmtrm_amount'].fillna('', inplace=True)
    df_fi_all['thstrm_add_amount'].fillna('', inplace=True)

    # 금액 컬럼에 있는 ''을 '0'로 변환(저장 시에 숫자 형식으로 변환을 위해서임)
    df_fi_all = df_fi_all.replace({'thstrm_amount' : '',
                                  'frmtrm_amount' : '',
                                  'bfefrmtrm_amount' : '',
                                  'thstrm_add_amount' : ''}, '0')
    
    # 타입 변환
    df_fi_all = df_fi_all.astype({'thstrm_amount' : 'float',
                                  'frmtrm_amount' : 'float',
                                  'bfefrmtrm_amount' : 'float',
                                  'thstrm_add_amount' : 'float'})
    
    df_fi_all = df_fi_all.astype({'ord' : 'int'})
    
    # df_fi_all.to_excel('D:\PythonProject\data-gatherer\dart\data\co_fi_all_multiple.xlsx')

### 3. 데이터베이스에 저장 (dart_report_detail)

In [32]:
if not df_fi_all.empty:
    conn = dbconnect.db_connect("DEMO_DW")
    cur = conn.cursor()
    # 테이블 컬럼 순서대로 항목 재정렬
    df_fi_save = df_fi_all[['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm', 'account_id', 'account_nm', 'account_detail', \
                        'thstrm_nm', 'thstrm_amount', 'thstrm_add_amount', 'frmtrm_nm', 'frmtrm_amount', 'bfefrmtrm_nm', 'bfefrmtrm_amount', \
                        'ord', 'currency']]
    # df_fi_save.to_excel('D:\PythonProject\data-gatherer\dart\data\df_fi_save.xlsx')

    for row in df_fi_save.itertuples():
        sql = "insert into dart_report_detail(rcept_no, reprt_code, bsns_year, corp_code, sj_div, sj_nm, account_id, account_nm, account_detail, \
                                            thstrm_nm, thstrm_amount, thstrm_add_amount, frmtrm_nm, frmtrm_amount, bfefrmtrm_nm, bfefrmtrm_amount, \
                                            ord, currency) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], \
                          row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18]))

    conn.commit()
    conn.close()